<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema2_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procesamiento de datos para su análisis y visualización

En este notebook vamos a importar de [EUROSTAT](https://ec.europa.eu/eurostat/web/main/data/database) datos de PIB y paro para los países de la UE y a procesarlos para poder, posteriormente, visualizarlos y analizarlos.

La gran ventaja de importar y procesar los datos en Python es que, al automatizar el proceso, resulta mucho más rápido volver a realizar la misma tarea. Además, tener el código escrito nos ayuda a identificar y a corregir errores, los cuales son muy difíciles de detectar si realizamos la manipulación de datos en Excel.

*Daniel Oto-Peralías*

### Importamos las librerías que necesitamos

In [1]:
# IMPORTAMOS LIBRERIAS
import pandas as pd
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 30)
import matplotlib.pyplot as plt

### PIB en volumen

In [2]:
# PIB en volúmen (Gross domestic product, volumes (teina011))
link_pib="https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/teina011?format=TSV&compressed=true"
pib = pd.read_csv(link_pib, compression='gzip', header=0, sep='\t')
pib # En el excel de EUROSTAT podemos comprobar que son tablas equivalentes.

,"freq,na_item,unit,geo\TIME_PERIOD",2021-Q3,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2
0,"Q,B1GQ,PCH_Q1_SCA,AT",4.4,-0.5,0.6,2.1,-0.2,-0.3,0.1,-1.1,-0.4,0.1,0.1,-0.4
1,"Q,B1GQ,PCH_Q1_SCA,BE",2.2,0.8,0.1 p,0.7 p,0.3 p,0.2 p,0.4 p,0.3 p,0.3 p,0.3 p,0.3 p,0.2 p
2,"Q,B1GQ,PCH_Q1_SCA,BG",2.0,1.5,0.7,0.8,0.6,0.6,0.3 p,0.4 p,0.5 p,0.5 p,0.5 p,0.5 p
3,"Q,B1GQ,PCH_Q1_SCA,CH",2.3,0.4,0.1,1.0,0.3,0.1,0.3,-0.2,0.2,0.3,0.5,0.7
4,"Q,B1GQ,PCH_Q1_SCA,CY",3.0,2.0,0.4,0.7,1.0,1.1,0.0,0.3,1.1,0.9,1.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,"Q,B1GQ,PCH_Q4_SCA,RS",8.1,7.7,4.3,4.0,1.3,0.8,0.9,1.5,3.7,4.0,4.3,4.0
109,"Q,B1GQ,PCH_Q4_SCA,SE",5.9,6.1,2.2,2.5,2.2,-0.6,0.9,-0.6,-0.5,-0.1,0.5,0.6
110,"Q,B1GQ,PCH_Q4_SCA,SI",5.8,9.5,4.9,4.2,1.6,0.8,1.3,1.8,1.9,2.4,1.7,0.8
111,"Q,B1GQ,PCH_Q4_SCA,SK",2.9,2.7,3.5,1.7,1.3,1.1,0.8,1.6,1.8,2.1,2.6,2.1


Vamos a hacer el siguiente procesamiento a los datos:
1. Dividimos la columna ```freq,na_item,unit,geo\TIME_PERIOD``` en cuatro columnas.
2. Seleccionamos solo la unidad "PCH_Q1_SCA" que significa "Percentage change, Annual variation rate, Seasonally and Calendar Adjusted.
3. Borramos los valores "p" y los ":".
4. Transponemos la tabla.
5. Usamos los nombres de países como nombre de las columnas.
6. Convertimos las columnas en formato numérico.
7. Establecemos el index de la tabla como fecha.
8. Guardamos los datos en un fichero csv.


In [ ]:
# 1. Dividimos la columna en tres.
# Hay que separar por la coma=> Es fácil porque las posiciones de las comas son fijas en este caso.
pib['na_item']=pib['na_item,unit,geo\\time'].str[:4] # Nótese el uso de la doble barra (!)
pib['unit']=pib['na_item,unit,geo\\time'].str[5:15]
pib['geo']=pib['na_item,unit,geo\\time'].str[16:]
pib.drop(columns='na_item,unit,geo\\time',inplace=True)
pib

In [ ]:
# 2. Seleccionamos solo la variable PCH_Q4_SCA
pib=pib.loc[pib['unit']=="PCH_Q4_SCA"]
pib

In [ ]:
# 3. Borramos los valores p y los :
for col in pib.columns:
  pib[col]=pib[col].replace("p","",regex=True) # Dejamos espacio adelante porque dicho espacio existe en los datos.
  pib[col]=pib[col].replace(":","",regex=True)
  pib[col]=pib[col].replace(" ","",regex=True) # por si hay algún espacio
pib

In [ ]:
# 4. Transponemos la tabla
pib=pib.transpose()
pib

In [ ]:
# 5. Usamos la fila de nombres de países como nombres de las columnas
nombre_col = pib.iloc[-1] # Los nombres de países están en la última fila
pib = pib.iloc[:-3] # aplicamos un filtrado para quitar las tres últimas filas
pib.columns = nombre_col # Establecemos el nuevo nombre de las columnas
pib

In [ ]:
# 6. Convertimos las columnas en formato numérico.
for col in pib.columns:
  pib[col]=pd.to_numeric(pib[col], errors='coerce')
pib

In [ ]:
#7. Establecemos el index de la tabla como fecha
pib.index=pib.index.str.replace(" ","",regex=True)
pib.index=pd.to_datetime(pib.index)
pib

In [ ]:
#8. Guardamos los datos en un fichero csv.
pib.to_csv("pib_volumen_EUROSTAT.csv")

Nótese que el archivo "pib_volumen_EUROSTAT.csv" se ha guardado en la carpeta de archivos de Google Colab. Por tanto, tenemos que descargarla en nuestro ordenador.

### Tasa de desempleo

Repetimos los mismos pasos que anteriormente para el PIB, pero adaptando el código a las particularidades de la tabla de datos del paro.

In [ ]:
# Tasa de desempleo (Total unemployment rate (tps00203))
link_des="https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/une_rt_q.tsv.gz"
tp = pd.read_csv(link_des, compression='gzip', header=0, sep='\t')
tp

In [ ]:
# 1. Dividimos la columna en tres.
# Hay que separar por la coma=> Ojo, ahora es un poco más difícil
columna='s_adj,age,unit,sex,geo\\time'
tp['s_adj']=tp[columna].str.split(",").str[0]
tp['age']=tp[columna].str.split(",").str[1]
tp['unit']=tp[columna].str.split(",").str[2]
tp['sex']=tp[columna].str.split(",").str[3]
tp['geo']=tp[columna].str.split(",").str[4]
tp.drop(columns=columna,inplace=True)
tp

In [ ]:
# Exploramos un poco los datos
tp.unit.value_counts()


In [ ]:
# 2. Seleccionamos SA, Y15-74, PC_ACT, T
tp=tp.loc[(tp['s_adj']=="SA") &
          (tp['age']=="Y15-74") &
          (tp['unit']=="PC_ACT") &
          (tp['sex']=="T")]
tp

In [ ]:
# 3. Borramos los valores p y los :
for col in tp.columns:
  for c in ["p","d","u","b",":"," "]:
    tp[col]=tp[col].replace(c,"",regex=True)
tp

In [ ]:
# 4. Transponemos la tabla
tp=tp.transpose()
tp

In [ ]:
# 5. Usamos la fila de nombres de países como nombres de las columnas
nombre_col = tp.iloc[-1] # Los nombres de países están en la última fila
tp = tp.iloc[:-5] # aplicamos un filtrado para quitar las tres últimas filas
tp.columns = nombre_col # Establecemos el nuevo nombre de las columnas

# 6. Convertimos las columnas en formato numérico.
for col in tp.columns:
  tp[col]=pd.to_numeric(tp[col], errors='coerce')

#7. Establecemos el index de la tabla como fecha
tp.index=tp.index.str.replace(" ","",regex=True)
tp.index=pd.to_datetime(tp.index)
tp.sort_index(inplace=True)

#8. Guardamos los datos en un fichero csv.
tp.to_csv("Tasa_paro_EUROSTAT.csv")
